In [77]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.preprocessing import scale
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV, train_test_split

import hervpd as hp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [96]:
df = pd.read_csv( 'data/12features/combined.csv')
print(df.activity.unique())
df.describe()


['eat' 'focused-active' 'focused-passive' 'household-chores'
 'leisure-active' 'leisure-passive' 'movement' 'sleep' 'rest-active']


user        AVGNN        SDNN     MeanHR       MinHR       MaxHR  \
count  544.000000   544.000000  544.000000  544.00000  544.000000  544.000000   
mean     1.187500   758.600000   72.865018   81.37714   67.362033  108.560417   
std      0.913249   129.785003   32.960406   13.75457   10.849680   36.385572   
min      0.000000   475.730000   17.833000   51.62800   41.437000   58.117000   
25%      0.000000   670.362500   51.871500   73.23225   60.292750   91.470250   
50%      2.000000   750.545000   67.057000   79.94200   66.181500  100.520000   
75%      2.000000   819.307500   85.820000   89.50400   74.285250  112.580000   
max      2.000000  1162.200000  264.250000  126.12000  104.900000  550.320000   

            RMSSD       pNNxx         TINN      powerVLF       powerLF  \
count  544.000000  544.000000   544.000000    544.000000    544.000000   
mean    46.409227   20.082227   358.020221   2877.387599   1423.919671   
std     32.537407   18.316263   146.885899   3619.576847   1443.217366   
min      4.882500    0.000000    83.000000     85.654000     57.852000   
25%     27.438750    6.593075   258.000000    995.182500    591.307500   
50%     37.994500   15.417500   339.000000   1807.950000   1040.350000   
75%     54.762250   26.790500   426.250000   3304.250000   1697.950000   
max    240.470000   98.232000  1103.000000  29716.000000  12958.000000   

            powerHF   ratioHFLF  
count    544.000000  544.000000  
mean    1000.824001    2.753764  
std     1841.656665    2.296974  
min        7.977700    0.154680  
25%      258.917500    1.257000  
50%      512.090000    2.037350  
75%      960.045000    3.579975  
max    16103.000000   18.228000

### Apply scale to make all features go to mean = 0 and SD = 1. That will allow SVM kernels not to put too much weight on features with larger values.

In [97]:
df.iloc[:, 3:15] = df.iloc[:, 3:15].apply(lambda x: scale(x))
df.describe()

C:\Users\julia\Anaconda3\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


user         AVGNN          SDNN        MeanHR         MinHR  \
count  544.000000  5.440000e+02  5.440000e+02  5.440000e+02  5.440000e+02   
mean     1.187500 -3.485774e-16 -5.371520e-16 -3.424548e-16  7.265430e-17   
std      0.913249  1.000920e+00  1.000920e+00  1.000920e+00  1.000920e+00   
min      0.000000 -2.181534e+00 -1.671177e+00 -2.164846e+00 -2.391674e+00   
25%      0.000000 -6.805001e-01 -6.375176e-01 -5.927038e-01 -6.521657e-01   
50%      2.000000 -6.212130e-02 -1.763742e-01 -1.044351e-01 -1.089082e-01   
75%      2.000000  4.681849e-01  3.934085e-01  5.913918e-01  6.386906e-01   
max      2.000000  3.112621e+00  5.811856e+00  3.255939e+00  3.463007e+00   

              MaxHR         RMSSD         pNNxx          TINN      powerVLF  \
count  5.440000e+02  5.440000e+02  5.440000e+02  5.440000e+02  5.440000e+02   
mean  -1.136754e-16  1.836766e-17  5.571524e-17 -5.142945e-17  1.200020e-16   
std    1.000920e+00  1.000920e+00  1.000920e+00  1.000920e+00  1.000920e+00   
min   -1.387634e+00 -1.277451e+00 -1.097424e+00 -1.874062e+00 -7.719972e-01   
25%   -4.701286e-01 -5.835725e-01 -7.371355e-01 -6.815649e-01 -5.204856e-01   
50%   -2.211816e-01 -2.588550e-01 -2.549112e-01 -1.296089e-01 -2.957312e-01   
75%    1.105736e-01  2.569569e-01  3.665839e-01  4.649362e-01  1.180401e-01   
max    1.215224e+01  5.969725e+00  4.270615e+00  5.076494e+00  7.421673e+00   

            powerLF       powerHF     ratioHFLF  
count  5.440000e+02  5.440000e+02  5.440000e+02  
mean   7.918502e-17  2.061260e-17 -9.051175e-17  
std    1.000920e+00  1.000920e+00  1.000920e+00  
min   -9.474144e-01 -5.396012e-01 -1.132567e+00  
25%   -5.774449e-01 -4.032181e-01 -6.522240e-01  
50%   -2.660186e-01 -2.656216e-01 -3.121818e-01  
75%    1.900494e-01 -2.216294e-02  3.600263e-01  
max    7.999277e+00  8.207869e+00  6.742991e+00

### Test dataset will have 20% of examples, the other 80% will be for training (with cross-validation)

In [104]:
train, test = train_test_split(df, test_size=0.2)
print (len(train), len(test))

435 109


### 3 - Model selection:  Perform an exhaustive search, with k-fold cross-validation,  in the space parameter consisting of:
* $C = 10^{i}, com -1 \leq i \leq 6$, for both linear and RBF kernels
* $\gamma = 10^{i}, com -3 \leq i \leq 3$, for RBF kernel.

In [105]:
crossval = StratifiedShuffleSplit(n_splits=4, test_size=0.2)

c_range = np.logspace(-1, 2, 4) 
gamma_range = np.logspace(-2, 1, 4)

param_lin=dict(C=c_range)
param_rbf=dict(C=c_range, gamma=gamma_range)


In [106]:
grid_lin = GridSearchCV(svm.SVC(kernel='linear', cache_size=1000), param_grid=param_lin, cv=crossval)
grid_lin.fit(X=train.iloc[:, 3:15], y=train['activity'])

print("Kernel linear --- ")
print("Best params: %s with score %0.5f" % (grid_lin.best_params_, grid_lin.best_score_))

Kernel linear --- 
Best params: {'C': 100.0} with score 0.38506


In [107]:
grid_rbf = GridSearchCV(svm.SVC(kernel='rbf', cache_size=1000), param_grid=param_rbf, cv=crossval)
grid_rbf.fit(X=train.iloc[:, 3:15], y=train['activity'])

print("Kernel RBF --- ")
print("Best params: %s with score %0.5f" % (grid_rbf.best_params_, grid_rbf.best_score_))

Kernel RBF --- 
Best params: {'C': 100.0, 'gamma': 0.10000000000000001} with score 0.46552


In [108]:
clf1 = svm.SVC(kernel='linear', cache_size=1000, C=grid_lin.best_params_['C'])
clf1.fit(X=train.iloc[:, 3:15], y=train['activity'])
hpp.printResults(test['activity'].values, clf1.predict(test.iloc[:, 3:15]))

clf2 = svm.SVC(kernel='rbf', cache_size=1000, C=grid_rbf.best_params_['C'], gamma=grid_rbf.best_params_['gamma'])
clf2.fit(X=train.iloc[:, 3:15], y=train['activity'])
hpp.printResults(test['activity'].values, clf1.predict(test.iloc[:, 3:15]))

------
Got 42 out of 109 right! :)
------
Got 42 out of 109 right! :)


## Now, we can run the same steps above with multiple experiments! Time to have fun!!

In [114]:
lessactiv = hp.excludeActivities(df, ['sleep', 'rest-active', 'leisure-active'])
print(lessactiv.activity.unique())
hp.runFlow(lessactiv)

ju =  df.loc[df['user'] == 0]
hp.runFlow(ju)

ron = df.loc[df['user'] == 1]
hp.runFlow(ron)

edu = df.loc[df['user'] == 2]
hp.runFlow(edu)

['eat' 'focused-active' 'focused-passive' 'household-chores'
 'leisure-passive' 'movement']


C:\Users\julia\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


348 train examples and 88 test examples
Best params for linear kernel: {'C': 100.0} with score 0.40000
Best params for RBF kernel: {'C': 1.0, 'gamma': 0.10000000000000001} with score 0.46786
--- test results for linear kernel:
46 out of 88 right! :)
--- test results for RBF kernel:
46 out of 88 right! :)


C:\Users\julia\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


148 train examples and 37 test examples
Best params for linear kernel: {'C': 10.0} with score 0.55000
Best params for RBF kernel: {'C': 10.0, 'gamma': 0.10000000000000001} with score 0.50000
--- test results for linear kernel:
20 out of 37 right! :)
--- test results for RBF kernel:
20 out of 37 right! :)


C:\Users\julia\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


57 train examples and 15 test examples
Best params for linear kernel: {'C': 1.0} with score 0.64583
Best params for RBF kernel: {'C': 10.0, 'gamma': 0.01} with score 0.64583
--- test results for linear kernel:
10 out of 15 right! :)
--- test results for RBF kernel:


C:\Users\julia\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


10 out of 15 right! :)
229 train examples and 58 test examples
Best params for linear kernel: {'C': 1.0} with score 0.42391
Best params for RBF kernel: {'C': 10.0, 'gamma': 0.10000000000000001} with score 0.47826
--- test results for linear kernel:
23 out of 58 right! :)
--- test results for RBF kernel:
23 out of 58 right! :)


user         activity posture     AVGNN      SDNN    MeanHR     MinHR  \
0       0              eat     sit -0.588436  0.902062  0.477434  0.313013   
1       0              eat     sit -0.853271  1.001970  0.816542  0.572245   
2       0              eat     sit -0.696406 -0.057699  0.611476  0.746420   
3       0              eat     sit -0.603552 -0.466139  0.495845  0.516155   
4       0              eat     sit -1.129675 -0.653931  1.211101  1.422453   
5       0              eat     sit -1.078543 -0.869722  1.134620  1.089971   
6       0              eat     sit -1.437081 -1.093833  1.708121  1.935197   
7       0   focused-active     sit  1.968139  4.687049 -1.615068 -2.259751   
8       0   focused-active     sit  3.112621  4.832812 -2.164846 -2.391674   
9       0   focused-active     sit  2.713903  4.909338 -1.989979 -2.214270   
10      0   focused-active     sit  1.435153  2.248854 -1.299974 -1.463143   
11      0   focused-active     sit  1.088646  5.811856 -1.069220 -2.366765   
12      0   focused-active     sit -0.392162  0.947005  0.247335 -0.224825   
13      0   focused-active     sit -0.220259 -0.610384  0.058934  0.486818   
14      0   focused-active     sit -0.187405 -0.716458  0.024295  0.486818   
15      0   focused-active     sit -0.263061 -0.634587  0.104779  0.491708   
16      0   focused-active     sit  0.199282  0.777525 -0.355783 -0.882869   
17      0   focused-active     sit  0.251262 -0.453355 -0.403229  0.038651   
18      0   focused-active     sit  0.288743 -0.570208 -0.436921 -0.070946   
19      0   focused-active     sit  0.450158 -0.485210 -0.577440 -0.200470   
20      0   focused-active     sit  0.340954 -0.608927 -0.483130  0.017525   
21      0   focused-active     sit  0.721162  0.401334 -0.797788 -1.048187   
22      0   focused-active     sit  0.525120 -0.543971 -0.640241 -0.605093   
23      0   focused-active     sit -0.815251 -1.063040  0.765749  0.994766   
24      0   focused-active     sit -0.711831 -0.479957  0.631051  0.872991   
25      0   focused-active     sit -0.862449 -0.511296  0.828986  0.801126   
26      0   focused-active     sit  0.097944 -0.914878 -0.260963 -0.234235   
27      0   focused-active     sit -0.338177 -1.336772  0.186936  1.088034   
28      0  focused-passive     sit -0.035553  0.332097 -0.130996  0.214394   
29      0  focused-passive     sit  0.125554 -0.797690 -0.287161  0.283861   
..    ...              ...     ...       ...       ...       ...       ...   
155     0            sleep     lie  0.043188  1.101575 -0.208424 -0.452137   
156     0            sleep     lie  1.035664  0.700149 -1.031816 -1.060549   
157     0            sleep     lie  0.868927 -0.074340 -0.910509 -0.829915   
158     0            sleep     lie  0.547793 -0.384451 -0.659015 -0.976690   
159     0            sleep     lie  0.590056 -0.127361 -0.693581 -0.804638   
160     0            sleep     lie -0.256506  0.194806  0.097647 -0.728529   
161     0            sleep     lie -0.300234 -0.365441  0.145166  0.329988   
162     0            sleep     lie  0.106119 -0.453628 -0.268750 -0.295675   
163     0            sleep     lie  0.199667  0.190646 -0.356147 -0.203053   
164     0            sleep     lie  0.495428 -0.387731 -0.615572 -0.839325   
165     0            sleep     lie  0.641110 -0.198998 -0.734624 -0.770596   
166     0            sleep     lie  1.009828  1.187515 -1.013405 -1.069774   
167     0            sleep     lie  1.508649  1.715300 -1.346183 -1.492295   
168     0            sleep     lie  1.628882  0.938503 -1.419681 -1.277529   
169     0            sleep     lie -0.497356 -1.493285  0.368570  0.774557   
170     0            sleep     lie -0.491572 -1.483901  0.361730  1.021150   
171     0            sleep     lie -0.438667 -1.323987  0.300312  0.836828   
172     0            sleep     lie -0.252418 -1.226660  0.093354  0.544108   
173     0            sleep     lie -0.145451 -1.393376 -0.019440  0.701215   
174     0            sleep 